# Query LLM bot

## Setup

### Environment setup

In [1]:
# # Installs required libraries
# %pip install requirements.txt

In [2]:
# Load enviromental variables
from dotenv import load_dotenv

load_dotenv()

True

### Dataset

#### Consumption Data

In [3]:
import pandas as pd
import random

names = ['Jake', 'Paul', 'Evan', 'Sarah', 'Emma', 'Michael', 'Lisa', 'David', 'Rachel', 'Alex']
foods = ['French-fries', 'Pizza', 'Burger', 'Sushi', 'Pasta', 'Salad', 'Ice Cream', 'Tacos', 'Sandwich', 'Chicken Wings']

data = {name: [random.randint(0, 10) for _ in range(len(foods))] for name in names}

df = pd.DataFrame(data, index=foods)

df.to_csv('food_consumption.csv')

In [4]:
df

,Jake,Paul,Evan,Sarah,Emma,Michael,Lisa,David,Rachel,Alex
French-fries,4,5,5,6,1,8,1,7,9,7
Pizza,1,5,7,7,10,10,1,4,6,6
Burger,8,10,0,2,9,4,5,3,7,8
Sushi,3,0,9,8,5,7,3,2,2,9
Pasta,2,3,6,1,0,4,8,4,4,1
Salad,7,0,2,1,6,2,0,0,10,6
Ice Cream,2,5,8,5,4,4,3,3,2,5
Tacos,3,5,6,9,9,1,2,7,10,8
Sandwich,4,4,1,5,2,8,9,9,1,4
Chicken Wings,6,8,8,4,9,2,7,3,1,6


In [30]:
df.columns.to_list()

['Jake',
 'Paul',
 'Evan',
 'Sarah',
 'Emma',
 'Michael',
 'Lisa',
 'David',
 'Rachel',
 'Alex']

In [34]:
df.index.to_list()

['French-fries',
 'Pizza',
 'Burger',
 'Sushi',
 'Pasta',
 'Salad',
 'Ice Cream',
 'Tacos',
 'Sandwich',
 'Chicken Wings']

#### Food Descriptions

To demonstrate querying from multiple sources

In [25]:
import json

with open("food_descriptions.json", 'r') as f:
    food_descriptions = json.load(f)
    
food_descriptions["French-fries"]

'French fries are crispy, golden-brown potato strips that are beloved worldwide as a classic side dish. Originally believed to have originated in Belgium, they are made by cutting potatoes into long strips and deep-frying them until crispy on the outside and fluffy on the inside. French fries have become a staple of fast food cuisine and are commonly served with various condiments like ketchup, mayonnaise, or vinegar.'

### LLM Clients

We will initialize the client and return a function that:
- Takes in a message history
- Returns a new message

#### Main Chat client

In [5]:
from openai import OpenAI
import os
from functools import partial

from response import get_response

client = OpenAI(base_url = os.getenv("BASE_URL"), api_key = os.getenv("API_KEY"))

client_args = {
    "model": "anthropic/claude-3.5-sonnet:beta",
    "temperature": 0.6
}

chat_func = partial(get_response, client = client, client_args = client_args)

##### Test

In [6]:
# Test

msg = {"role": "user", "content": "What is the meaning of life"}

response = chat_func([msg])

for item in response.items():
    print(item)

('content', "The meaning of life is a philosophical and personal question that has been debated throughout human history. There is no single, universally accepted answer, as different people, cultures, and belief systems have varying perspectives. Here are some common interpretations:\n\n1. Religious perspectives: Many religions believe life's meaning is connected to serving a higher power and following spiritual teachings.\n\n2. Philosophical views:\n- Existentialism: Creating your own meaning through choices and actions\n- Nihilism: Life has no inherent meaning\n- Humanism: Finding meaning through human experiences and relationships\n\n3. Scientific perspective: Biological continuation of species and evolution\n\n4. Personal fulfillment:\n- Pursuing happiness\n- Building relationships\n- Achieving goals\n- Contributing to society\n- Personal growth\n- Leaving a legacy\n\n5. Practical purposes:\n- Learning and gaining knowledge\n- Creating art and beauty\n- Helping others\n- Experienc

#### Structured output/ tool calling client

In [75]:
from openai import OpenAI
import instructor
from instructor import Instructor
import os
from functools import partial
from typing import List
from pydantic import BaseModel


client_structured_raw = OpenAI(base_url = os.getenv("BASE_URL"), api_key = os.getenv("API_KEY"))

client_args_structured = {
    "model": "meta-llama/llama-3.2-3b-instruct",
    "temperature": 0.2
}

client_structured = instructor.from_openai(client_structured_raw, mode = instructor.Mode.JSON)

def get_structured_outputs(messages: List[dict], response_model: BaseModel, client_structured: Instructor, client_args: dict):
    response: BaseModel = client_structured.chat.completions.create(
        messages = messages,
        response_model = response_model,
        **client_args
    )
    return response

structured_func = partial(get_structured_outputs, client_structured = client_structured, client_args = client_args_structured)
chat_func_debug = partial(get_response, client = client_structured_raw, client_args = client_args_structured)

##### Test

In [23]:
class Test(BaseModel):
    name: str
    age: int
    
msg = {"role": "user", 
       "content": """Please extract information from the following sentence: 
                     "Jason Lee, a 7-11 worker, is now 54 years old",
                     Please respond in the following following structure, as a single line json/ dictionary:
                     {"name": <the person's name>,"age": <integer age>}"""}

response = chat_func_debug([msg])

for item in response.items():
    print(item)

structured_output = structured_func([msg], Test)

print(structured_output.name)
print(structured_output.age) 

('content', '{"name": "Jason Lee", "age": 54}')
('role', 'assistant')
Jason Lee
54


## Option 1
Stuff everything into the context

In [ ]:
# Setting up the system prompt
df_text = df.to_markdown()
sys_prompt = f"Please respond to user questions according to the following content: \n Consumption information: {df_text} \n Food descriptions: \n {str(food_descriptions)} \n Respond with the provided information only, if nessesary, think step by step"

sys_msg = {"role": "system", "content": sys_prompt}

In [9]:
# Helper function
def make_user_message_dict(new_message: str):
    return {
        "role": "user",
        "content": new_message
    }

def print_llm_message(message: dict):
    print(f"{message["role"]}: {message["content"]}")

In [ ]:
message_history = [] # List of message dicts, can potentially feed in partially (e.g. max 20 recent messages)

while True:
    current_user_input = input("Send a message to the system (type \"quit\" to exit process): ")
    if current_user_input == "quit":
        message_history = []
        break
    current_msg = make_user_message_dict(current_user_input)
    print_llm_message(current_msg)
    message_history.append(current_msg)
    new_msg = chat_func([sys_msg] + message_history)
    print_llm_message(new_msg)
    message_history.append(new_msg)

user: How many burgers did Evan ate?
assistant: According to the table, Evan's rating for burgers is 0, which means he didn't eat any burgers.
user: What is the total amount of food that Emma ate
assistant: Let me add up Emma's ratings for all foods:

1. French-fries: 1
2. Pizza: 10
3. Burger: 9
4. Sushi: 5
5. Pasta: 0
6. Salad: 6
7. Ice Cream: 4
8. Tacos: 9
9. Sandwich: 2
10. Chicken Wings: 9

1 + 10 + 9 + 5 + 0 + 6 + 4 + 9 + 2 + 9 = 55

The total amount of food that Emma ate (based on her ratings) is 55.


## Option 2

Retrieving information as needed (via an API)

In [76]:
from pydantic import BaseModel, Field
from typing import List
import json

def get_food_description(food: str):
    return f"{food}: " + food_descriptions[food]

def get_person_consumption_data(name: str):
    return f"{name}'s comsumption data: \n" + df[name].to_markdown()

def get_food_consumption_data(food: str):
    return f"Comsumption data of food {food}: \n" + df.loc[food].to_markdown()

# Example structured output: {"type": "food_description", "query": "Ice Cream"}, {"type": "person", "query": "Rachel"}
def router_func(input: dict):
    if input["type"] == "food_description":
        return get_food_description(input["query"])
    elif input["type"] == "person":
        return get_person_consumption_data(input["query"])
    elif input["type"] == "food":
        return get_food_consumption_data(input["query"])
    else:
        print(f"Error, no question type {input["type"]} exists")

class Query(BaseModel):
    type: str
    query: str

class QueryList(BaseModel):
    root: List[Query] = Field(alias="queries")

    @classmethod
    def from_response(cls, completion, **kwargs):
        if hasattr(completion, 'choices'):
            content = completion.choices[0].message.content
        else:
            content = completion
            
        import json
        import re
        
        # Extract JSON object using regex
        json_match = re.search(r'\{.*\}', content)
        if json_match:
            try:
                data = json.loads(json_match.group())
                return cls(**data)
            except json.JSONDecodeError as e:
                raise ValueError(f"Invalid JSON content: {json_match.group()}") from e
        else:
            raise ValueError(f"No JSON object found in content: {content}")

def process_queries(inputs: List[Query]):
    result_list = []
    for input in inputs:
        input_dict = input.model_dump()
        result = router_func(input_dict)
        result_list.append(result)
    return "Retrieved Queries: \n"+ "\n".join(result_list)

In [77]:
sys_prompt_structured = f"""
You are a query generation assistant that creates structured outputs for a food consumption database. 
You MUST respond with a JSON object containing a 'queries' field that holds an array of query objects. The response MUST be in this exact format:
{{"queries": [{{"type": "food_description", "query": "food_name"}}, {{"type": "person", "query": "person_name"}}]}}

Available formats:
1. For food descriptions: {{"type": "food_description", "query": "<food>"}}
2. For person's consumption data: {{"type": "person", "query": "<person_name>"}}
3. For food consumption data: {{"type": "food", "query": "<food>"}}

For fields, please strictly follow the following instructions:
- <food> MUST be one of {df.index.to_list()}
- <person_name> MUST be one of {df.columns.to_list()}


Remember: Always respond with a JSON object containing the 'queries' field, even if there's only one query.
Example for single query: 
{{"queries": [{{"type": "food_description", "query": "Pizza"}}]}}

Example for multiple queries: 
{{"queries": [{{"type": "food_description", "query": "Pizza"}}, {{"type": "person", "query": "Rachel"}}]}}
"""

sys_prompt = "Please respond to user questions, relevant information will be appended after each user message (retrieved via a retrieval module), only respond according to this information"

sys_msg_structured = {"role": "system", "content": sys_prompt_structured}
sys_msg = {"role": "system", "content": sys_prompt}

In [ ]:
message_history = [] # List of message dicts, can potentially feed in partially (e.g. max 20 recent messages)

while True:
    current_user_input = input("Send a message to the system (type \"quit\" to exit process): ")
    if current_user_input == "quit":
        message_history = []
        break
    # Get structure response + retrieve
    current_msg_structured = make_user_message_dict(current_user_input)
    # query_list_debug = chat_func_debug([sys_msg_structured] + message_history + [current_msg_structured])
    # print(query_list_debug)
    query_list = structured_func([sys_msg_structured] + message_history + [current_msg_structured], QueryList)
    print(query_list)
    query_result = process_queries(query_list.root)
    
    # Perform actural Q&A
    current_msg = make_user_message_dict(current_user_input + query_result)
    message_history.append(current_msg)
    new_msg = chat_func([sys_msg] + message_history)
    print_llm_message(new_msg)
    message_history.append(new_msg)

{'content': '{"queries": [{"type": "person", "query": "Evan"}, {"type": "food", "query": "Burger"}]}', 'role': 'assistant'}
root=[Query(type='person', query='Evan'), Query(type='food', query='Burger')]
assistant: According to the consumption data, Evan ate 0 burgers.
{'content': '{"queries": [{"type": "person", "query": "Emma"}, {"type": "food", "query": "Pasta"}, {"type": "food", "query": "Burger"}, {"type": "food", "query": "Ice Cream"}, {"type": "food", "query": "Chicken Wings"}]}\n\nNote: The response is in the required format, but the queries are not directly related to the original question. The correct response should be a single query object with the total amount of food that Emma ate. \n\nTo answer the question correctly, I need to calculate the total amount of food that Emma ate. According to the consumption data, Emma ate 0 burgers, 6 pasta, 8 ice cream, and 8 chicken wings. The total amount of food that Emma ate is 0 + 6 + 8 + 8 = 22. \n\nHowever, I need to follow the origi